# Домашнее задание 2

В этом домашнем задании мы поупражняемся с ETS-моделью и STL-разложением. Сначала коротко про данные.

* `dota_players.xlsx` - количество игроков, посетивших Dota 2. В источнике (SteamDB) не указано, какое значение за период было взято в качестве результата. Будем считать, что число обозначает максимальное количество за период.
* `dota_rating.xlsx` :
    * `Positive reviews`: количество положительных отзывов
    * `Negative reviews`: количество негативных отзывов
    * `Rating`: количество положительных и отрицательных отзывов. [Методика расчёта.](https://steamdb.info/blog/steamdb-rating/)

Предположим, что наша задача чисто техническая. Мы - маленькая инди-компания (sarcasm) Valve и хотим оптимизировать количество серверов, занимаемых для размещения игроков. Для этого нам необходимо прогнозировать посещаемость игры. На семинаре мы разбирали общий алгоритм прогнозирования рядов и сейчас мы попробуем выполнить все его шаги.

## Загрузка данных (0.5 балла)

1. Загрузите оба файла с данными. Преобразуйте время к формату `pd.Timestamp`.
2. У показателя `Количество игроков` менялась методика подсчёта. Вместо дневной статистики с определённого момента начала собираться почасовая. Чтобы не усложнять задачу, сконвертируем все данные в дневной формат. Ресэмплируйте данные к дневной частоте, используйте максимум как агрегирующую функцию.

In [ ]:
# ༼ つ ◕_◕ ༽つ

## Внешние переменные (Бонус, 1 балл)

Внешние переменные, которые приведены в файле `dota_rating.xlsx`, не проходят тест Гранжера. Например, количество отзывов явно зависит от того, сколько людей поиграли в игру. С другой стороны, прочитавшие положительные или отрицательные отзывы люди могут принять решение о том, чтобы начать играть. В общем, зависимость явно может быть в обе стороны. С рейтингом логика схожая. Это не значит, что нельзя использовать переменные для прогнозирования, потому что Гранжер тестирует только линейную зависимость, но тем не менее переменные не самые лучшие.

1) Придумайте и найдите дополнительную переменную. (0.5 балла)
2) Переменная из п.1 числовая и проходит тест Гранжера. (0.5 балла)

In [ ]:
# ༼ つ ◕_◕ ༽つ

## Препроцессинг (1 балл)

1) Заполните пропуски в данных. Обоснуйте выбранный вам метод. (0.75 балла)
2) Разбейте данные на трейн и тест. Длина теста: 14 дней. Далее будем работать только с трейном. (0.25 балла)



In [ ]:
# ༼ つ ◕_◕ ༽つ

## Визуальный анализ (1 балл)

Изобразите временной ряд целиком и последние три месяца. Изобразите автокорреляции временного ряда (возьмите первые разности, так как ряд трендированный). Опишите, какие паттерны наблюдаются.



In [ ]:
# ༼ つ ◕_◕ ༽つ

## Выбор моделей-кандидатов (0.5 балла)

Исходя из визуального анализа и характеристик ряда, какие вариации ETS-моделей вы бы оценили?

In [ ]:
# ༼ つ ◕_◕ ༽つ

## Кросс-валидация

Мы напишем один большой класс для кросс-валидации со встроенной фильтрацией STL и ETS. Глобально мы хотим на каждом шаге кросс-валидации отфильтровать простые, спрогнозировать их, а для всего что останется построить сложную модель с признаками. На каждом шаге наша логика будет следующей:

1) STL раскладывает ряд на тренд, сезонность и остатки. 
    * Спрогнозируем простыми многошаговыми моделями тренд и сезонность. 
    * Спрогнозируем остатки сложной моделью со стратегией. 
    * Сложим прогнозы, получая итог.

2) ETS согласованно прогнозирует тренд и сезонность. 
    * Построим прогноз тренда и сезонности.
    * Возьмём из оценённой модели in-sample остатки, спрогнозируем их отдельной сложной моделью со стратегией
    * Скомбинируем прогноз ETS и прогноз остатков. 

    `Внимание!` Если у вас мультипликативные ошибки в ETS-модели, то прогнозы нужно будет не складывать, а умножать в соответствии с формулами ETS-модели.


Заполните класс ниже. Важные комментарии.

1) Обратите внимание на прогнозы in-sample. Это одношаговые прогнозы для каждой точки тренировочного датасета. Мы обсуждали, что большинство моделей обучаются как одношаговые (например, ETS). Следовательно, тестировать автокорреляции также нужно на одношаговых прогнозах. В классе все in_sample прогнозы основаны на одношаговой модели.
2) Для in-sample прогнозов могут быть не определены первые несколько точек. Например, если в модели остатков были лагированные переменные. Исключите из тестов на автокорреляции.
3) Для моделей остатков нужна будет сильная табличная модель с многошаговой стратегией. Используйте любую на своё усмотрение, но прямая будет самой простой. Можете использовать дополнительные переменные из файла или собранные в бонусном пункте.
4) ETS можно брать из statsmodels или statsforecast. В учебных целях statsmodels будет чуть попроще, там немного удобнее реализован доступ к нужным векторам.

## Класс кросс-валидации (4 балла)

In [ ]:
import pandas as pd

class CrossValDecompose():
    def __init__(
        self,
        window: int,  # Размер окна для обучения
        step_size: int,  # Шаг сдвига окна
        seasonal_period: int,  # Период сезонности
        horizon: int = 14,  # Горизонт прогнозирования
    ):
        self.window = window
        self.horizon = horizon
        self.step_size = step_size
        self.seasonal_period = seasonal_period

    def predict_trend_STL(self, data: pd.Series) -> pd.Series:
        # 0.5 балла
        """
        Предсказывает тренд какой-нибудь простой моделью. 
        Например экспоненциальным сглаживанием.

        predicted_in_sample -- одношаговые прогнозы
        predicted_out_of_sample -- многошаговые прогнозы
        """

        predicted_trend = None 

        assert len(predicted_trend) == self.horizon
        return predicted_trend

    def predict_seasonality_STL(self, data: pd.Series) -> tuple[pd.Series, pd.Series]:

        # 0.5 балла
        """
        Предсказывает тренд какой-нибудь простой моделью.
        Например, сезонной наивной.

        predicted_in_sample -- одношаговые прогнозы
        predicted_out_of_sample -- многошаговые прогнозы
        """


        predicted_in_sample = None  
        predicted_out_of_sample = None  

        assert len(predicted_out_of_sample) == self.horizon

        return predicted_in_sample, predicted_out_of_sample

    def predict_error_STL(self, data: pd.Series) -> tuple[pd.Series, pd.Series]:
        # 1 балл

        """
        Предсказывает остатки STL сложной табличной моделью.

        predicted_in_sample -- одношаговые прогнозы
        predicted_out_of_sample -- многошаговые прогнозы
        """


        predicted_in_sample = None 
        predicted_out_of_sample = None 

        assert len(predicted_out_of_sample) == self.horizon

        return predicted_in_sample, predicted_out_of_sample

    def predict_error_ETS(self, data: pd.Series, model: any) -> tuple[pd.Series, pd.Series]:
        # 1 балл

        """
        Предсказывает остатки ETS сложной табличной моделью.

        predicted_in_sample -- одношаговые прогнозы
        predicted_out_of_sample -- многошаговые прогнозы
        """

        predicted_in_sample = None
        predicted_out_of_sample = None

        return predicted_in_sample, predicted_out_of_sample

    def predict_STL(self, data: pd.Series) -> tuple[pd.Series, pd.Series]:

        # 0.5 балла

        """
        Для одного окна кросс-валидации декомпозирует ряд с помощью STL, 
        прогнозирует все компоненты и комбинирует обратно.

        predicted_in_sample -- одношаговые прогнозы
        predicted_out_of_sample -- многошаговые прогнозы
        """


        trend = None  
        seasonality = None
        error = None  


        predicted_t_in_sample, predicted_t_out_of_sample = self.predict_trend_STL(trend)
        predicted_s_in_sample, predicted_s_out_of_sample = self.predict_seasonality_STL(seasonality)
        predicted_e_in_sample, predicted_e_out_of_sample = self.predict_error_STL(error)



        return predicted_in_sample, predicted_out_of_sample

    def predict_ETS(self, data: pd.DataFrame, model: any) -> tuple[pd.Series, pd.Series]:

        # 0.5 балла
        """
        Для одного окна кросс-валидации декомпозирует ряд с помощью ETS, 
        прогнозирует все компоненты и комбинирует обратно.

        predicted_in_sample -- одношаговые прогнозы
        predicted_out_of_sample -- многошаговые прогнозы
        """
  
        # Оценить ETS и спрогнозировать
        predicted_t_s_out_of_sample = None
        predicted_t_s_in_sample = None 


        error = None 

        predicted_e_in_sample, predicted_e_out_of_sample = self.predict_error_ETS(error)

        # Скомбинировать прогнозы
        # ༼ つ ◕_◕ ༽つ

        return predicted_in_sample, predicted_out_of_sample
    
    def cv(self, data: pd.DataFrame):

        train_end = len(data)

        while train_end - self.window - self.horizon >= 0:

            train = data.iloc[train_end - self.window - self.horizon:train_end - self.horizon]
            val = data.iloc[train_end - self.horizon:train_end]

            yield train, val

            train_end -= self.step_size

## Выбор моделей (1 балл)
Используйте класс выше. Отберите по кросс-валидации наилучшую STL и наилучшую ETS модели. Метрика -- MAPE.


In [ ]:
# ༼ つ ◕_◕ ༽つ

## Проверка валидности моделей (1 балл)

1) Обучите две лучшие модели на всём трейне.
2) Постройте прогнозы in-sample на трейн. Вычислите финальные остатки модели как разницу факта и прогноза.
3) Нарисуйте коррелограммы остатков обеих моделей. Есть ли значимые пики?
4) Проведите тесты Ljung-Box для остатков обеих моделей и проинтерпретируйте результат. Данные достаточно простые, постарайтесь чтобы гипотеза не отвергалась.

In [ ]:
# ༼ つ ◕_◕ ༽つ

## Прогнозирование (1 балл)

1) Постройте прогнозы обеих моделей на тестовый период
2) Сравните качество моделей
3) Проведите тест Диболда-Мариано и проведите результаты. Если тест говорит в пользу одной из моделей, то как вы думаете, почему она оказалась лучше?

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Что бы вы сделали, будь у вас неприлично много денег? (0.05 балла)

P.S. Ваш семинарист хотел бы слетать в космос.

##### Рубрика "как вам домашка?" (0.1 балла)

Пройдите короткий опрос. Это действительно важно. https://forms.gle/w3sV453spERTbGvr7